In [2]:
#Importing necesarry libraries
import pandas as pd 
import seaborn as sns
from wordcloud import WordCloud
import matplotlib.pyplot as plt
import nltk
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from sklearn.model_selection import train_test_split
import tensorflow.compat.v2 as tf
import pandas as pd
import numpy as np 
import warnings
warnings.filterwarnings('ignore')
import re
import string
import pickle

In [3]:
#importing dataset
data = pd.read_csv(r'C:\Users\ASUS\Documents\Python\data.csv', encoding='latin', names = ['polarity','id','date','query','user','text'])

In [4]:
data = data.sample(frac=1)
data = data[:200000]

In [5]:
print("Dataset shape:", data.shape)

Dataset shape: (200000, 6)


In [6]:
data.head(10)

,polarity,id,date,query,user,text
1308994,4,2012954704,Tue Jun 02 21:37:00 PDT 2009,NO_QUERY,lindsayrcg,thinking about laying down. pretty sure maggie...
1156126,4,1979067708,Sun May 31 01:30:24 PDT 2009,NO_QUERY,bestnileyfans,"@JonasVideos hey, thanks for following us. Tha..."
1360524,4,2048719161,Fri Jun 05 15:39:42 PDT 2009,NO_QUERY,carillee,Going to Lido's for dinner.
541612,0,2200148421,Tue Jun 16 18:30:17 PDT 2009,NO_QUERY,dianaily0,Going to romania. Pretty scared.
301220,0,1998590560,Mon Jun 01 18:48:11 PDT 2009,NO_QUERY,marii85,Can't sleep testing twitter from my mobile
505774,0,2188476369,Mon Jun 15 21:58:16 PDT 2009,NO_QUERY,FaithDani,Just loves how @justinjeremy NEVER tweets back...
1234363,4,1992375287,Mon Jun 01 08:39:07 PDT 2009,NO_QUERY,pusscat30,is basking in the sun
1191119,4,1983830281,Sun May 31 13:55:10 PDT 2009,NO_QUERY,redrobinrockn,@VickiElam U should meet @StatiK99 cause he's ...
898330,4,1693583396,Sun May 03 23:02:01 PDT 2009,NO_QUERY,karleexx,400th tweet!! yay
1183352,4,1982415840,Sun May 31 11:09:50 PDT 2009,NO_QUERY,viridian,Guess who finally has a new Blackberry?


In [7]:
data['polarity'].unique()

array([4, 0], dtype=int64)

In [8]:
# Replacing the value 4 to 1.
data['polarity'] = data['polarity'].replace(4,1)
data.head()

,polarity,id,date,query,user,text
1308994,1,2012954704,Tue Jun 02 21:37:00 PDT 2009,NO_QUERY,lindsayrcg,thinking about laying down. pretty sure maggie...
1156126,1,1979067708,Sun May 31 01:30:24 PDT 2009,NO_QUERY,bestnileyfans,"@JonasVideos hey, thanks for following us. Tha..."
1360524,1,2048719161,Fri Jun 05 15:39:42 PDT 2009,NO_QUERY,carillee,Going to Lido's for dinner.
541612,0,2200148421,Tue Jun 16 18:30:17 PDT 2009,NO_QUERY,dianaily0,Going to romania. Pretty scared.
301220,0,1998590560,Mon Jun 01 18:48:11 PDT 2009,NO_QUERY,marii85,Can't sleep testing twitter from my mobile


In [9]:
data.describe()

,polarity,id
count,200000.000000,2.000000e+05
mean,0.502180,1.997965e+09
std,0.499996,1.934211e+08
min,0.000000,1.467814e+09
25%,0.000000,1.956658e+09
50%,1.000000,2.001829e+09
75%,1.000000,2.176809e+09
max,1.000000,2.329205e+09


In [10]:
# checking positive and negative labeled tweets
positives = data['polarity'][data.polarity == 1 ]
negatives = data['polarity'][data.polarity == 0 ]

print('Total length of the data is:         {}'.format(data.shape[0]))
print('No. of positve tagged sentences is:  {}'.format(len(positives)))
print('No. of negative tagged sentences is: {}'.format(len(negatives)))

Total length of the data is:         200000
No. of positve tagged sentences is:  100436
No. of negative tagged sentences is: 99564


In [11]:
def word_count(words):
    return len(words.split())

In [20]:
from collections import Counter
all_words = []
for line in list(data['text']):
    words = line.split()
    for word in words:
      if(len(word)>2):
        all_words.append(word.lower())
    
    
Counter(all_words).most_common(20)

[('the', 64758),
 ('and', 37208),
 ('you', 29842),
 ('for', 26467),
 ('have', 17946),
 ('that', 16109),
 ("i'm", 15889),
 ('but', 15561),
 ('just', 15353),
 ('with', 14167),
 ('was', 12740),
 ('not', 12715),
 ('this', 11148),
 ('get', 10028),
 ('good', 9758),
 ('are', 9555),
 ('like', 9383),
 ('all', 9278),
 ('out', 8649),
 ('your', 8234)]

In [21]:
# dropping unnecesarry columns
data.drop(['date','query','user','word count'], axis=1, inplace=True)

In [22]:
data.drop('id', axis=1, inplace=True)

In [23]:
data.head(10)

,polarity,text
1308994,1,thinking about laying down. pretty sure maggie...
1156126,1,"@JonasVideos hey, thanks for following us. Tha..."
1360524,1,Going to Lido's for dinner.
541612,0,Going to romania. Pretty scared.
301220,0,Can't sleep testing twitter from my mobile
505774,0,Just loves how @justinjeremy NEVER tweets back...
1234363,1,is basking in the sun
1191119,1,@VickiElam U should meet @StatiK99 cause he's ...
898330,1,400th tweet!! yay
1183352,1,Guess who finally has a new Blackberry?


In [24]:
#Checking if any null values present
(data.isnull().sum() / len(data))*100

polarity    0.0
text        0.0
dtype: float64

In [25]:
#convrting pandas object to a string
data['text'] = data['text'].astype('str')

In [26]:
nltk.download('stopwords')
stopword = set(stopwords.words('english'))
print(stopword)

{"mightn't", "you're", 'y', 'weren', 'didn', 'shouldn', "doesn't", 'here', 'your', 'yours', 'into', 'no', 'our', 'any', 'over', 'with', 'when', 'themselves', "needn't", 'these', "don't", 'shan', 'during', 'i', 'those', 'most', 'her', 'all', 'or', 'doing', 'we', 'as', "she's", 's', 'ours', 've', "should've", 'while', 'of', 'through', 'ma', 'up', 'isn', 'from', 'again', 'are', 'am', 'they', 'until', 'hers', "haven't", 'more', 'because', "you've", "shouldn't", 'him', 'what', 'you', 'both', 'them', 'only', 'very', 'who', 'will', "isn't", 'each', 'has', "hasn't", 'too', 'this', 'in', 'just', 'then', 'should', 'd', 'did', 'itself', 'out', 'me', 'aren', 'that', 'after', 'yourself', 'off', 'same', 're', 'own', 'ain', 'for', 'before', 'myself', 'some', 'whom', 'on', 'above', 'which', 'he', 'were', 'why', 'so', 'down', 'once', "shan't", "it's", 'o', 'hasn', 'do', 'couldn', "you'll", 'be', 'needn', 'an', 'other', 'don', 'was', "mustn't", "wouldn't", 'than', 'its', 'being', 'and', 'by', 'below', '

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\ASUS\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [27]:
nltk.download('punkt')
nltk.download('wordnet')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\ASUS\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\ASUS\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [28]:
urlPattern = r"((http://)[^ ]*|(https://)[^ ]*|( www\.)[^ ]*)"
userPattern = '@[^\s]+'
def process_tweets(tweet):
  # Lower Casing
    tweet = tweet.lower()
    tweet=tweet[1:]
    # Removing all URls 
    tweet = re.sub(urlPattern,'',tweet)
    # Removing all @username.
    tweet = re.sub(userPattern,'', tweet) 
    #Remove punctuations
    tweet = tweet.translate(str.maketrans("","",string.punctuation))
    #tokenizing words
    tokens = word_tokenize(tweet)
    #Removing Stop Words
    final_tokens = [w for w in tokens if w not in stopword]
    #reducing a word to its word stem 
    wordLemm = WordNetLemmatizer()
    finalwords=[]
    for w in final_tokens:
      if len(w)>1:
        word = wordLemm.lemmatize(w)
        finalwords.append(word)
    return ' '.join(finalwords)

In [25]:
data.head(10)

,polarity,text,processed_tweets
97918,0,@danecook really wanted to go to that show. co...,danecook really wanted go show couldnt afford ...
406827,0,Great. The power is off. And I have wet hair t...,reat power wet hair need dried date night boy
646821,0,@Orchidflower yes it would be - she'd taken th...,orchidflower yes would shed taken dog walk amp...
1060028,1,@trixr4kedzz they are cool bags and @dancingsh...,trixr4kedzz cool bag friend great bag great se...
1236164,1,@A_Phelly then I'll follow you into the dark ...,aphelly ill follow dark im talented finishing ...
408233,0,is at my cuzin Amber's grad party but still a ...,cuzin amber grad party still little upset morning
1075008,1,@florinda_3rs Lucky @trishheylady. *stomping f...,florinda3rs lucky stomping foot blogger fun be...
463545,0,@AndrePaular that's awesome. I would donate bu...,andrepaular thats awesome would donate havent ...
747388,0,Alright so the Taste of Chicago starts this co...,lright taste chicago start coming weekendif pl...
334409,0,People come on..why do you keep deleting then ...,eople come onwhy keep deleting refollowing im ...


In [39]:
from keras.models import Sequential
from keras.layers import Dense, Embedding, LSTM, Dropout
from sklearn.feature_extraction.text import CountVectorizer
from keras.preprocessing.text import Tokenizer
from keras_preprocessing.sequence import pad_sequences
from keras.utils.np_utils import to_categorical
import re

In [40]:
import keras
keras.__version__

'2.10.0'

In [41]:
from keras.preprocessing.text import Tokenizer
from keras_preprocessing.sequence import pad_sequences
from keras import regularizers

max_words = 5000
max_len = 200
#tokenizer of keras
tokenizer = Tokenizer(num_words=max_words)
tokenizer.fit_on_texts(data.processed_tweets)
sequences = tokenizer.texts_to_sequences(data.processed_tweets)
tweets = pad_sequences(sequences, maxlen=max_len)
print(tweets)

[[   0    0    0 ...  590  156  114]
 [   0    0    0 ...  593   22  246]
 [   0    0    0 ...   26 1743   91]
 ...
 [   0    0    0 ...   64   25  440]
 [   0    0    0 ...  229   22 2572]
 [   0    0    0 ... 1513  136   62]]


In [42]:
X_train, X_test, y_train, y_test = train_test_split(tweets, data.polarity.values, test_size=0.2, random_state=101)

### Adding LSTM to the process input allows RNN to remember its inputs for a long time, which is useful for learning long-term dependencies. In this case, the word embedding is used to generate a numerical vector representation of each word, which is then fed into the LSTM layer. We’ve used keras tokeniser and pad sequences to turn words into tokens and sequences, which we then used to generate word embeddings. We have used three activation functions as hidden layers. Two of them are Relu, and the last one is Sigmoid activation function.

In [43]:
from keras.models import Sequential
from keras import layers
from keras import regularizers
from keras import backend as K
from keras.callbacks import ModelCheckpoint
model2 = Sequential()
model2.add(layers.Embedding(max_words, 128))
model2.add(layers.LSTM(64,dropout=0.5))
model2.add(layers.Dense(16, activation='relu'))
model2.add(layers.Dense(8, activation='relu'))
model2.add(layers.Dense(1,activation='sigmoid'))
model2.compile(optimizer='adam',loss='binary_crossentropy', metrics=['accuracy'])
checkpoint2 = ModelCheckpoint("rnn_model.hdf5", monitor='val_accuracy', verbose=1,save_best_only=True, mode='auto', period=1,save_weights_only=False)
history = model2.fit(X_train, y_train, epochs=10,validation_data=(X_test, y_test),callbacks=[checkpoint2])


Epoch 1/10
5000/5000 [==============================] - ETA: 0s - loss: 0.5066 - accuracy: 0.7479
Epoch 1: val_accuracy improved from -inf to 0.76998, saving model to rnn_model.hdf5
5000/5000 [==============================] - 915s 182ms/step - loss: 0.5066 - accuracy: 0.7479 - val_loss: 0.4771 - val_accuracy: 0.7700
Epoch 2/10
5000/5000 [==============================] - ETA: 0s - loss: 0.4672 - accuracy: 0.7749
Epoch 2: val_accuracy improved from 0.76998 to 0.77363, saving model to rnn_model.hdf5
5000/5000 [==============================] - 960s 192ms/step - loss: 0.4672 - accuracy: 0.7749 - val_loss: 0.4715 - val_accuracy: 0.7736
Epoch 3/10
5000/5000 [==============================] - ETA: 0s - loss: 0.4518 - accuracy: 0.7832
Epoch 3: val_accuracy improved from 0.77363 to 0.77387, saving model to rnn_model.hdf5
5000/5000 [==============================] - 699s 140ms/step - loss: 0.4518 - accuracy: 0.7832 - val_loss: 0.4706 - val_accuracy: 0.7739
Epoch 4/10
5000/5000 [===============

In [50]:
sequence = tokenizer.texts_to_sequences(['today is my birhtday. had a good day.'])
test = pad_sequences(sequence, maxlen=max_len)
pred = model2.predict(test)
if pred > 0.5:
  print('Positive')
else:
  print('Negative')
# print(pred)

1/1 [==============================] - 0s 331ms/step
Positive


In [45]:
model = keras.models.load_model('rnn_model.hdf5')
sequence = tokenizer.texts_to_sequences([' the day is very good'])
test = pad_sequences(sequence, maxlen=max_len)
pred = model.predict(test)
if pred > 0.5:
  print('Positive')
else:
  print('Negative')

1/1 [==============================] - 1s 682ms/step
Positive


In [48]:
sequence = tokenizer.texts_to_sequences(['I am feeling bad today'])
test = pad_sequences(sequence, maxlen=max_len)
pred = model.predict(test)
if pred > 0.5:
  print('Positive')
else:
  print('Negative')

1/1 [==============================] - 0s 114ms/step
Negative
